# Visualization reducing grammar

## Inspecting the corpora

1. Sentence length
2. Vocabsize
3. Amount of terminals
4. Amount of non terminals
5. Distribution (Zipf)
6. Train/test/dev

#### Per top k

In [2]:
# open txt file
version = 'normal'
top_k = '0.2'
with open(f'corpora/{version}/train_sent_{top_k}.txt', 'r') as file:
    data = [item.strip('\n')

In [3]:
data

'I grabbed his head and felt a long look on the window .I saw a great breath on the water , taking it on the ground .I took a deep place into the road , looking on the door .`` I <apostrophe>ll have been made , <apostrophe><apostrophe> he said .I grabbed his face and smiled .`` I do n<apostrophe>t want to do him , <apostrophe><apostrophe> he said .I said and shook his face .I opened his hand and smiled .I went up on the room , taking it on the ground .`` I <apostrophe>ll have given up in the side , <apostrophe><apostrophe> he said .I opened his face , looking into the door .I opened his head and opened his face .I shook his face , holding it on the table .`` I <apostrophe>ll be gone , <apostrophe><apostrophe> he said .I took a big man into the road and opened his head .I said and took a long thing into the water .I took his hand on the ground , coming on the door .I grabbed his head , making his face into the floor .I said and shook his head .`` I <apostrophe>ll have had a bit of light